### Import Library

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Parameters

In [29]:
#adjust values here
user_num = 5
alpha = 0.001
batch_size = 32
global_rounds = 50
local_epochs = 1
learning_rate = 1e-3

### Model

In [16]:
#model 
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.01)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.01)
        x = F.softmax(self.fc3(x))
        return x

### Set Train and Test dataset 

In [17]:
train_dataset = pd.read_csv('BankChurners_fdtrain.csv')
test_dataset = pd.read_csv('BankChurners_fdtest.csv')

train_y = train_dataset['CreditLevel']
train_x = train_dataset.drop(columns=['CreditLevel','CustomerId'])
train_x = train_x.to_numpy()
train_y= train_y.to_numpy()
train_x = torch.Tensor(train_x) 
train_y = torch.Tensor(train_y)
train_set = TensorDataset(train_x,train_y)

In [18]:
test_y = test_dataset['CreditLevel']
test_x = test_dataset.drop(columns=['CreditLevel','CustomerId'])
test_x = test_x.to_numpy()
test_y= test_y.to_numpy()
test_x = torch.Tensor(test_x) 
test_y = torch.Tensor(test_y)
test_set = TensorDataset(test_x,test_y)

### Dirichlet Partition

In [19]:
def dirichlet_partition(training_data, testing_data, alpha, user_num):
    idxs_train = np.arange(len(training_data))
    idxs_valid = np.arange(len(testing_data))

    if hasattr(training_data, 'CreditLevel'):
        labels_train = training_data.CreditLevel
        labels_valid = testing_data.CreditLevel

    idxs_labels_train = np.vstack((idxs_train, labels_train))
    idxs_labels_train = idxs_labels_train[:, idxs_labels_train[1,:].argsort()]
    idxs_labels_valid = np.vstack((idxs_valid, labels_valid))
    idxs_labels_valid = idxs_labels_valid[:, idxs_labels_valid[1,:].argsort()]

    labels = np.unique(labels_train, axis=0)

    data_train_dict = data_organize(idxs_labels_train, labels)
    data_valid_dict = data_organize(idxs_labels_valid, labels)

    data_partition_profile_train = {}
    data_partition_profile_valid = {}


    for i in range(user_num):
        data_partition_profile_train[i] = []
        data_partition_profile_valid[i] = []

    ## Distribute rest data
    for label in data_train_dict:
        proportions = np.random.dirichlet(np.repeat(alpha, user_num))
        proportions_train = len(data_train_dict[label])*proportions
        proportions_valid = len(data_valid_dict[label]) * proportions

        for user in data_partition_profile_train:

            data_partition_profile_train[user]   \
                = set.union(set(np.random.choice(data_train_dict[label], int(proportions_train[user]) , replace = False)), data_partition_profile_train[user])
            data_train_dict[label] = list(set(data_train_dict[label])-data_partition_profile_train[user])


            data_partition_profile_valid[user] = set.union(set(
                np.random.choice(data_valid_dict[label], int(proportions_valid[user]),
                                 replace=False)), data_partition_profile_valid[user])
            data_valid_dict[label] = list(set(data_valid_dict[label]) - data_partition_profile_valid[user])


        while len(data_train_dict[label]) != 0:
            rest_data = data_train_dict[label][0]
            user = np.random.randint(0, user_num)
            data_partition_profile_train[user].add(rest_data)
            data_train_dict[label].remove(rest_data)

        while len(data_valid_dict[label]) != 0:
            rest_data = data_valid_dict[label][0]
            user = np.random.randint(0, user_num)
            data_partition_profile_valid[user].add(rest_data)
            data_valid_dict[label].remove(rest_data)

    for user in data_partition_profile_train:
        data_partition_profile_train[user] = list(data_partition_profile_train[user])
        data_partition_profile_valid[user] = list(data_partition_profile_valid[user])
        np.random.shuffle(data_partition_profile_train[user])
        np.random.shuffle(data_partition_profile_valid[user])

    return data_partition_profile_train, data_partition_profile_valid


def data_organize(idxs_labels, labels):
    data_dict = {}

    labels = np.unique(labels, axis=0)
    for one in labels:
        data_dict[one] = []

    for i in range(len(idxs_labels[1, :])):
        data_dict[idxs_labels[1, i]].append(idxs_labels[0, i])
    return data_dict

In [20]:
train_index, test_index = dirichlet_partition(train_dataset, test_dataset, alpha=alpha, user_num=user_num)

In [21]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        inputs, label = self.dataset[self.idxs[item]]
        
        return torch.tensor(inputs), torch.tensor(label)
    
train_data_list = []
for user_index in range(user_num):
    train_data_list.append(DatasetSplit(train_set, train_index[user_index]))

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
global_model = NN().to(device)

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

### Local Training with FedProx Equation Added

In [33]:
def local_trainer(dataset, model, global_round, device, local_epoch, batchsize):

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    criterion = nn.CrossEntropyLoss().to(device)
    model.train()
    epoch_loss = []
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    ##FedProx parameters
    proximal_term = 0
    mu=1e-4
    global_round_now=0
    
    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (inputs, labels) in enumerate(dataloader):  

            inputs = inputs.to(device)
            labels = torch.tensor(labels,dtype = torch.long)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            
            #Check if global round has changed
            #If changed, update proximal term
            if global_round_now != global_round:
                global_model = copy.deepcopy(model)
                for w, w_t in zip(model.parameters(), global_model.parameters()) :
                    proximal_term += (w-w_t).norm(2)
            global_round_now = global_round
            ## FedProx Loss
            loss = criterion(outputs, labels) + (mu/2)*proximal_term

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

            if batch_idx % 10 == 0:
                print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    global_round, iter, batch_idx * len(inputs),
                    len(dataloader.dataset),
                    100. * batch_idx / len(dataloader), loss.item()))
            batch_loss.append(loss.item())
        epoch_loss.append(sum(batch_loss)/len(batch_loss))
    return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

### Weight Averaging

In [24]:
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key].float(), len(w))
    return w_avg

### Model Inference

In [25]:
def inference(model, testloader):
    """ Returns the inference accuracy and loss.
    """
    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)
    loss, total, correct = 0.0, 0.0, 0.0
    for batch_idx, (inputs, labels) in enumerate(testloader):
        labels = torch.tensor(labels,dtype = torch.long)
        inputs, labels = inputs.to(device), labels.to(device)

        # Inference
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        correct += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
    loss /= batch_idx
    accuracy = correct/total
    return accuracy, loss

### Model Training

In [30]:
global_value = []

for round_idx in range(global_rounds):
    local_weights = []
    local_losses = []
    global_acc = []

    for user_index in range(user_num):
        model_weights, loss = local_trainer(train_data_list[user_index], copy.deepcopy(global_model), round_idx, device, local_epochs, batch_size)
        local_weights.append(copy.deepcopy(model_weights))
        local_losses.append(loss)

    global_weight = average_weights(local_weights)
    global_model.load_state_dict(global_weight)
    test_acc, test_loss = inference(global_model, test_loader)
    print('Global Round :{}, the global accuracy is {:.3}%, and the global loss is {:.3}.'.format(round_idx, 100 * test_acc, test_loss))
    global_value.append(test_acc)

/tmp/ipykernel_437/1651168755.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(inputs), torch.tensor(label)
/tmp/ipykernel_437/2057983531.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)
/tmp/ipykernel_437/2676379489.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


| Global Round : 0 | Local Epoch : 0 | [0/3799 (0%)]	Loss: 2.274338
| Global Round : 0 | Local Epoch : 0 | [320/3799 (8%)]	Loss: 2.059807
| Global Round : 0 | Local Epoch : 0 | [640/3799 (17%)]	Loss: 2.101989
| Global Round : 0 | Local Epoch : 0 | [960/3799 (25%)]	Loss: 2.013201
| Global Round : 0 | Local Epoch : 0 | [1280/3799 (34%)]	Loss: 2.042106
| Global Round : 0 | Local Epoch : 0 | [1600/3799 (42%)]	Loss: 2.054796
| Global Round : 0 | Local Epoch : 0 | [1920/3799 (50%)]	Loss: 2.054996
| Global Round : 0 | Local Epoch : 0 | [2240/3799 (59%)]	Loss: 2.035871
| Global Round : 0 | Local Epoch : 0 | [2560/3799 (67%)]	Loss: 2.001022
| Global Round : 0 | Local Epoch : 0 | [2880/3799 (76%)]	Loss: 2.039533
| Global Round : 0 | Local Epoch : 0 | [3200/3799 (84%)]	Loss: 2.038430
| Global Round : 0 | Local Epoch : 0 | [3520/3799 (92%)]	Loss: 2.003028
| Global Round : 0 | Local Epoch : 0 | [0/1266 (0%)]	Loss: 2.370751
| Global Round : 0 | Local Epoch : 0 | [320/1266 (25%)]	Loss: 2.093177
| Glo

/tmp/ipykernel_437/2600621470.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)


| Global Round : 1 | Local Epoch : 0 | [1280/3799 (34%)]	Loss: 2.043519
| Global Round : 1 | Local Epoch : 0 | [1600/3799 (42%)]	Loss: 2.054014
| Global Round : 1 | Local Epoch : 0 | [1920/3799 (50%)]	Loss: 2.051361
| Global Round : 1 | Local Epoch : 0 | [2240/3799 (59%)]	Loss: 2.033743
| Global Round : 1 | Local Epoch : 0 | [2560/3799 (67%)]	Loss: 1.998980
| Global Round : 1 | Local Epoch : 0 | [2880/3799 (76%)]	Loss: 2.040574
| Global Round : 1 | Local Epoch : 0 | [3200/3799 (84%)]	Loss: 2.032653
| Global Round : 1 | Local Epoch : 0 | [3520/3799 (92%)]	Loss: 2.000263
| Global Round : 1 | Local Epoch : 0 | [0/1266 (0%)]	Loss: 2.364468
| Global Round : 1 | Local Epoch : 0 | [320/1266 (25%)]	Loss: 2.020022
| Global Round : 1 | Local Epoch : 0 | [640/1266 (50%)]	Loss: 1.547673
| Global Round : 1 | Local Epoch : 0 | [960/1266 (75%)]	Loss: 1.475596
| Global Round : 1 | Local Epoch : 0 | [0/2531 (0%)]	Loss: 2.305047
| Global Round : 1 | Local Epoch : 0 | [320/2531 (12%)]	Loss: 1.960040
| Gl

--END--